In [16]:
import pandas as pd
from geopy.distance import geodesic

exporter_data = pd.read_csv('./../../../dataset/district/merged_district_with_names.csv')
terminal_locations = pd.read_csv('./../../../dataset/district/terminal_locations.csv')

cochin_port = (9.9575, 76.2711)  # Latitude, Longitude of Cochin port

# Function to calculate distance between two points
def calculate_distance(row):
    exporter_location = (row['Latitude'], row['Longitude'])
    return geodesic(cochin_port, exporter_location).kilometers

# Calculate distance from Cochin port
exporter_data['Distance from Cochin port'] = exporter_data.apply(calculate_distance, axis=1)

# Function to find closest terminal
def find_closest_terminal(row):
    exporter_location = (row['Latitude'], row['Longitude'])
    distances = terminal_locations.apply(lambda x: geodesic((x['Latitude'], x['Longitude']), exporter_location).kilometers, axis=1)
    closest_terminal_index = distances.idxmin()
    closest_terminal = terminal_locations.loc[closest_terminal_index]
    return closest_terminal['Terminal'], distances[closest_terminal_index]

# Find closest terminal for each exporter
exporter_data['Name of the closest terminal'], exporter_data['Distance from NW3 closest terminal'] = zip(*exporter_data.apply(find_closest_terminal, axis=1))

new_dataset = exporter_data[['Name', 'Distance from Cochin port', 'Distance from NW3 closest terminal', 'Name of the closest terminal']]


new_dataset.to_csv('results.csv', index=False)
